In [227]:
from langgraph.graph import StateGraph, END
from dotenv import load_dotenv
import os
from langgraph.prebuilt import tool_node
import google.generativeai as genai
import fitz
from typing import TypedDict
from pprint import pprint
import json
# from langchain_google_genai import ChatGoogleGenerativeAI

In [228]:
class AgentType(TypedDict):
    offer_text: str
    offer_analysis: dict

In [229]:
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")

In [230]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text() for page in doc)
    return text

offer_text = extract_text_from_pdf("offer.pdf")
print(offer_text[:1000])

 
EMPLOYMENT OFFER     
 
 
 
Date: 4th May 2017. 
To, 
 
Ms.Archana Tarle, 
Nasik 
Sub: Offer letter for the post of Software Engineer. 
Dear Ms. Archana Tarle, 
With reference to the personal interview you had with us on 19th April 2017, we hereby offer you the 
post of Software Engineer with our Organization. You should join Netwin latest by 18th May 2017 at 
Nasik by 10.30 am. 
You will be governed by the rules and regulations of the Company. You will get the Letter of 
Appointment after receiving your acceptance of this offer letter and the necessary documents as 
mentioned in this letter. 
You will be paid gross emoluments as detailed in Annexure – A. 
Your employment with us will be governed by the Terms & Conditions as detailed in Annexure – B. 
This Offer Letter is subject to the following conditions: 
  1) Netwin should receive your acceptance letter within 2 days of issue of this letter. 
  2) You should submit to Netwin a copy of your Resignation Letter given to your curren

In [231]:
def offer_analyzer_agent(state: AgentType) -> AgentType:
    offer_text = state["offer_text"]
    prompt = f"""
    You are an HR assistant. Extract structured information from the following job offer letter:

    Text:
    {offer_text}

    Return in JSON format with fields:
    - job_title
    - salary
    - location
    - joining_date
    - bond_terms
    """
    response = model.generate_content(prompt)
    
    return {"offer_analysis": response.text}


In [232]:
builder = StateGraph(AgentType)


builder.add_node("OfferAnalyzer", offer_analyzer_agent)
builder.set_entry_point("OfferAnalyzer")
builder.set_finish_point("OfferAnalyzer")

graph = builder.compile()


In [233]:
result = graph.invoke({
    "offer_text": offer_text
})

json_string = result["offer_analysis"].strip().replace('```json\n', '').replace('\n```', '')
formatted_dict = json.loads(json_string)

pprint(formatted_dict)


{'bond_terms': None,
 'job_title': 'Software Engineer',
 'joining_date': '2017-05-18',
 'location': 'Nasik',
 'salary': {'gross_annual': 314935.38,
            'monthly_breakdown': {'basic': 8000,
                                  'bonus': 6300,
                                  'conveyance': 3600,
                                  'education_allowance': 3600,
                                  'gratuity': 4615,
                                  'hra': 4800,
                                  'leave_benefit': 14000,
                                  'medical_reimbursement': 1500,
                                  'mediclaim': 1000,
                                  'pf': 960,
                                  'variable': 3000},
            'variable_component': 'Based on monthly PPI (Personal Performance '
                                  'Index) + CPI (Company Performance Index)'}}
